# Pilot 2 -- Compute the baseline decodability of various input stimuli combinations
### Ex: Color stims ([red, red] v. [red, blue] v. [blue, red] v. [blue, blue])

## Use SVM classifications to input stimuli (both modalities: visual and auditory)
## Using Ciric-style postprocessing

## Takuya Ito
#### 12/06/2018

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns
import h5py
os.sys.path.append('glmScripts/')
import taskGLMPipeline as tgp

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [3]:
def loadInputActivity(subj,inputtype):
    x = tgp.loadTaskTiming(subj,'ALL')
    stimIndex = np.asarray(x['stimIndex'])
    ind = np.where(stimIndex==inputtype)[0]
    
    datadir = basedir + 'data/postProcessing/hcpPostProcCiric/'
    h5f = h5py.File(datadir + subj + '_glmOutput_data.h5','r')
    data = h5f['taskRegression/ALL_24pXaCompCorXVolterra_taskReg_betas_canonical'][:].copy()
    data = data[:,ind].copy()
    h5f.close()
    return data


# Localize relevant regions for each sensory rule using univariate activation analyses

In [4]:
sensoryActivations = np.zeros((len(glasser2),4,len(subjNums)))
scount = 0
for subj in subjNums:
    sensoryActivations[:,:,scount] = loadInputActivity(subj,'Sensory')
    scount += 1

In [5]:
roiActivity = np.zeros((nParcels,sensoryActivations.shape[1]))
sensoryRules = ['srRed','srVertical','srHigh','srConstant']
for roi in range(nParcels):
    roi_ind = np.where(glasser2==roi+1)[0]
    rulecount = 0
    for rule in sensoryRules:
        roi_activity = np.mean(sensoryActivations[roi_ind,rulecount,:],axis=0)
        t, p = stats.ttest_1samp(roi_activity,0)
        roiActivity[roi,rulecount] = p
        rulecount += 1

activeROIs = {}
rulecount = 0
for rule in sensoryRules:
    h0, q = mc.fdrcorrection0(roiActivity[:,rulecount])
    roiActivity[:,rulecount] = h0
    print 'Number of significant activations for', rule, 'rule:', np.sum(h0)
    activeROIs[rule] = np.where(h0)[0] + 1
    rulecount += 1
    
        

Number of significant activations for srRed rule: 276
Number of significant activations for srVertical rule: 284
Number of significant activations for srHigh rule: 290
Number of significant activations for srConstant rule: 291


# 0.1 Define functions for SR decodings

In [6]:
from scipy.spatial.distance import cdist

def srDecoding(data, inputtype, rois=np.arange(1,nParcels+1), ncvs=100, nproc=5):
    """
    Run an across-subject classification
    Decode input stimuli (4-way decoding) for each sensory rule type
    Limit to ROIs within either vis or aud network
    """
        
    nstim = 4 # Number of stimuli per stimulus-type
    
    nSubjs = data.shape[2]
#     if inputtype in ['cproColorStim','cproOriStim']:
#         # Visual networks
#         rois = np.where((networkdef==networkmappings['vis1']) | (networkdef==networkmappings['vis2']))[0] 
#         rois = rois + 1
#     elif inputtype in ['cproConstantStim','cproPitchStim']:
#         rois = np.where(networkdef==networkmappings['aud'])[0]
#         rois = rois + 1
#     rois = np.arange(nParcels)+1
    netStats = np.zeros((len(rois),nstim*nSubjs))


    nsamples = nSubjs * nstim

    # Label array for supervised learning
    labels = np.tile(range(nstim),nSubjs)
    subjarray = np.repeat(range(nSubjs),nstim)

    # Run SVM classifications on network-level activation patterns across subjects
    roicount = 0
    for roi in rois:
        print 'Computing SVM classification for input stimuli:', inputtype, 'for ROI', roicount, '/', len(rois)
        roi_ind = np.where(glasser2==roi)[0]
        nfeatures = len(roi_ind)
        roi_ind.shape = (len(roi_ind),1)       

        svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
        samplecount = 0
        scount = 0
        for subj in range(len(subjNums)):
            roidata = np.squeeze(data[roi_ind,:,scount])
            svm_mat[samplecount:(samplecount+nstim),:] = roidata.T

            scount += 1
            samplecount += nstim

#             # Spatially demean matrix across features
#             samplemean = np.mean(svm_mat,axis=1)
#             samplemean.shape = (len(samplemean),1)
#             svm_mat = svm_mat - samplemean

        scores = randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=nproc)
        netStats[roicount,:] = scores
        roicount += 1


    return netStats

def randomSplitLOOBaselineCV(ncvs, svm_mat, labels, subjarray, nproc=5):
    """
    Runs cross validation for an across-subject SVM analysis
    """
    
    ntasks = len(np.unique(labels))
    nsamples = svm_mat.shape[0]
    nsubjs = nsamples/ntasks

    subjects = np.unique(subjarray)
    indices = np.arange(nsamples)
    
    numsubjs_perfold = 8
    if nsubjs%numsubjs_perfold!=0: 
        raise Exception("Error: Folds don't match number of subjects")
        
    nfolds = nsubjs/numsubjs_perfold
    subj_array_folds = subjarray.copy()
    
    inputs = [] 
    
    for fold in range(nfolds):
        test_subjs = np.random.choice(subj_array_folds,numsubjs_perfold,replace=False)
        train_subjs_all = np.delete(subjects,test_subjs)
        for cv in range(ncvs):
            # Randomly sample half of train set subjects for each cv (CV bootstrapping)
            train_subjs = np.random.choice(train_subjs_all,
                                         int(np.floor(len(train_subjs_all)*(2.0))),
                                         replace=True)

            train_ind = []
            for subj in train_subjs:
                train_ind.extend(np.where(subjarray==subj)[0])

            test_ind = []
            for subj in test_subjs:
                test_ind.extend(np.where(subjarray==subj)[0])
            
            train_ind = np.asarray(train_ind)
            test_ind = np.asarray(test_ind)

            trainset = svm_mat[train_ind,:]
            testset = svm_mat[test_ind,:]

            # Normalize trainset and testset
            mean = np.mean(svm_mat[train_ind,:],axis=0)
            mean.shape = (1,len(mean))
            std = np.std(svm_mat[train_ind,:],axis=0)
            std.shape = (1,len(std))

            trainset = np.divide((trainset - mean),std)
            testset = np.divide((testset - mean),std)

            inputs.append((trainset,testset,labels[train_ind],labels[test_ind]))
        
        subj_array_folds = np.delete(subj_array_folds,test_subjs)
        
    pool = mp.Pool(processes=nproc)
    scores = pool.map_async(_decoding,inputs).get()
    pool.close()
    pool.join()
    
#     subj_acc = np.zeros((len(subjects),))
#     scount = 0
#     i = 0
#     for subj in subjects:
#         subjmean = []
#         for cv in range(ncvs):
#             subjmean.append(scores[i])
#             i += 1
        
#         subj_acc[scount] = np.mean(subjmean)
        
#         scount += 1

#     return subj_acc
    acc = []
    for score in scores:
        acc.extend(score)
    return acc

def _decoding((trainset,testset,trainlabels,testlabels)):

#     clf = sklearn.linear_model.LogisticRegression()
    clf = svm.SVC(C=1.0, kernel='linear')

    clf.fit(trainset,trainlabels)
    predictions = clf.predict(testset)
    acc = predictions==testlabels
#     score = np.mean(acc)
    
    return acc

    

# 1.0 Run across subject decoding on RED SR rules
### 4-way decoding
* both red
* notboth red
* either red
* neither red

## 1.1 Load data for RED SR integration

In [7]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srRed'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

In [8]:
nproc = 30
ncvs = 1
rois = activeROIs[sr_rule]


# distances_baseline = srDecoding(data_task, sr_rule,rois=activeROIs[sr_rule], ncvs=ncvs, nproc=nproc)
distances_baseline = srDecoding(data_task, sr_rule,rois=rois, ncvs=ncvs, nproc=nproc)


Computing SVM classification for input stimuli: srRed for ROI 0 / 276
Computing SVM classification for input stimuli: srRed for ROI 1 / 276
Computing SVM classification for input stimuli: srRed for ROI 2 / 276
Computing SVM classification for input stimuli: srRed for ROI 3 / 276
Computing SVM classification for input stimuli: srRed for ROI 4 / 276
Computing SVM classification for input stimuli: srRed for ROI 5 / 276
Computing SVM classification for input stimuli: srRed for ROI 6 / 276
Computing SVM classification for input stimuli: srRed for ROI 7 / 276
Computing SVM classification for input stimuli: srRed for ROI 8 / 276
Computing SVM classification for input stimuli: srRed for ROI 9 / 276
Computing SVM classification for input stimuli: srRed for ROI 10 / 276
Computing SVM classification for input stimuli: srRed for ROI 11 / 276
Computing SVM classification for input stimuli: srRed for ROI 12 / 276
Computing SVM classification for input stimuli: srRed for ROI 13 / 276
Computing SVM cl

## 1.2 Compute statistics

In [9]:

nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]
    


    

In [10]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for srRed Stimuli: 4
Accuracies: [ 0.35677083  0.3515625   0.32291667  0.33072917]


## 1.3 Map accuracies back to cortical surface

In [11]:
# Put all data into a single matrix (since we only run a single classification)
srStats = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    srStats[vertex_ind,0] = statistics[roicount,0]
    srStats[vertex_ind,1] = statistics[roicount,1]
    srStats[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Decoding'
np.savetxt(outdir + filename + '.csv', srStats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

# 2.0 Run across subject decoding on VERTICAL SR rules
### 4-way decoding
* both vertical
* notboth vertical
* either vertical
* neither vertical

## 1.1 Load data for VERTICAL SR integration

In [12]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srVertical'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

In [13]:
nproc = 30
ncvs = 1
rois = activeROIs[sr_rule]

# distances_baseline = srDecoding(data_task, sr_rule,rois=activeROIs[sr_rule], ncvs=ncvs, nproc=nproc)
distances_baseline = srDecoding(data_task, sr_rule,rois=rois, ncvs=ncvs, nproc=nproc)



Computing SVM classification for input stimuli: srVertical for ROI 0 / 284
Computing SVM classification for input stimuli: srVertical for ROI 1 / 284
Computing SVM classification for input stimuli: srVertical for ROI 2 / 284
Computing SVM classification for input stimuli: srVertical for ROI 3 / 284
Computing SVM classification for input stimuli: srVertical for ROI 4 / 284
Computing SVM classification for input stimuli: srVertical for ROI 5 / 284
Computing SVM classification for input stimuli: srVertical for ROI 6 / 284
Computing SVM classification for input stimuli: srVertical for ROI 7 / 284
Computing SVM classification for input stimuli: srVertical for ROI 8 / 284
Computing SVM classification for input stimuli: srVertical for ROI 9 / 284
Computing SVM classification for input stimuli: srVertical for ROI 10 / 284
Computing SVM classification for input stimuli: srVertical for ROI 11 / 284
Computing SVM classification for input stimuli: srVertical for ROI 12 / 284
Computing SVM classifi

## 1.2 Compute statistics

In [14]:

nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]
    


    

In [15]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for srVertical Stimuli: 2
Accuracies: [ 0.328125  0.328125]


## 1.3 Map accuracies back to cortical surface

In [16]:
# Put all data into a single matrix (since we only run a single classification)
srStats = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    srStats[vertex_ind,0] = statistics[roicount,0]
    srStats[vertex_ind,1] = statistics[roicount,1]
    srStats[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Decoding'
np.savetxt(outdir + filename + '.csv', srStats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

# 3.0 Run across subject decoding on HIGH SR rules
### 4-way decoding
* both high
* notboth high
* either high
* neither high

## 1.1 Load data for HIGH SR integration

In [17]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srHigh'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

In [18]:
nproc = 30
ncvs = 1
rois = activeROIs[sr_rule]

# distances_baseline = srDecoding(data_task, sr_rule,rois=activeROIs[sr_rule], ncvs=ncvs, nproc=nproc)
distances_baseline = srDecoding(data_task, sr_rule,rois=rois, ncvs=ncvs, nproc=nproc)



Computing SVM classification for input stimuli: srHigh for ROI 0 / 290
Computing SVM classification for input stimuli: srHigh for ROI 1 / 290
Computing SVM classification for input stimuli: srHigh for ROI 2 / 290
Computing SVM classification for input stimuli: srHigh for ROI 3 / 290
Computing SVM classification for input stimuli: srHigh for ROI 4 / 290
Computing SVM classification for input stimuli: srHigh for ROI 5 / 290
Computing SVM classification for input stimuli: srHigh for ROI 6 / 290
Computing SVM classification for input stimuli: srHigh for ROI 7 / 290
Computing SVM classification for input stimuli: srHigh for ROI 8 / 290
Computing SVM classification for input stimuli: srHigh for ROI 9 / 290
Computing SVM classification for input stimuli: srHigh for ROI 10 / 290
Computing SVM classification for input stimuli: srHigh for ROI 11 / 290
Computing SVM classification for input stimuli: srHigh for ROI 12 / 290
Computing SVM classification for input stimuli: srHigh for ROI 13 / 290
Co

## 1.2 Compute statistics

In [19]:

nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]
    


    

In [20]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for srHigh Stimuli: 5
Accuracies: [ 0.3359375   0.33072917  0.32552083  0.33072917  0.32291667]


## 1.3 Map accuracies back to cortical surface

In [21]:
# Put all data into a single matrix (since we only run a single classification)
srStats = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    srStats[vertex_ind,0] = statistics[roicount,0]
    srStats[vertex_ind,1] = statistics[roicount,1]
    srStats[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Decoding'
np.savetxt(outdir + filename + '.csv', srStats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0

# 4.0 Run across subject decoding on CONSTANT SR rules
### 4-way decoding
* both constant
* notboth constant
* either constant
* neither constant

## 1.1 Load data for CONSTANT SR integration

In [22]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
sr_rule = 'srConstant'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadInputActivity(subj,sr_rule)
    scount += 1

In [23]:
nproc = 30
ncvs = 1
rois = activeROIs[sr_rule]

# distances_baseline = srDecoding(data_task, sr_rule,rois=activeROIs[sr_rule], ncvs=ncvs, nproc=nproc)
distances_baseline = srDecoding(data_task, sr_rule,rois=rois, ncvs=ncvs, nproc=nproc)


Computing SVM classification for input stimuli: srConstant for ROI 0 / 291
Computing SVM classification for input stimuli: srConstant for ROI 1 / 291
Computing SVM classification for input stimuli: srConstant for ROI 2 / 291
Computing SVM classification for input stimuli: srConstant for ROI 3 / 291
Computing SVM classification for input stimuli: srConstant for ROI 4 / 291
Computing SVM classification for input stimuli: srConstant for ROI 5 / 291
Computing SVM classification for input stimuli: srConstant for ROI 6 / 291
Computing SVM classification for input stimuli: srConstant for ROI 7 / 291
Computing SVM classification for input stimuli: srConstant for ROI 8 / 291
Computing SVM classification for input stimuli: srConstant for ROI 9 / 291
Computing SVM classification for input stimuli: srConstant for ROI 10 / 291
Computing SVM classification for input stimuli: srConstant for ROI 11 / 291
Computing SVM classification for input stimuli: srConstant for ROI 12 / 291
Computing SVM classifi

## 1.2 Compute statistics

In [24]:

nStims = 4
statistics = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = len(subjNums)*nStims
    p = stats.binom_test(np.mean(distances_baseline[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics[roicount,0] = np.mean(distances_baseline[roicount,:])
    statistics[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics[:,1])
for roicount in range(len(rois)):
    statistics[roicount,1] = qs[roicount]
    statistics[roicount,2] = h0[roicount]*statistics[roicount,0]
    


    

In [25]:
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics[:,1]<0.05)[0]
print 'Number of ROIs significant for', sr_rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics[sig_ind,0]


Number of ROIs significant for srConstant Stimuli: 2
Accuracies: [ 0.3359375  0.328125 ]


## 1.3 Map accuracies back to cortical surface

In [26]:
# Put all data into a single matrix (since we only run a single classification)
srStats = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi)[0]
    srStats[vertex_ind,0] = statistics[roicount,0]
    srStats[vertex_ind,1] = statistics[roicount,1]
    srStats[vertex_ind,2] = statistics[roicount,2]

    roicount += 1

    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/Decoding_SR/'
filename = sr_rule + '_SR_Decoding'
np.savetxt(outdir + filename + '.csv', srStats,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)




0